In [14]:
import json, os, csv

In [15]:
# filepath to the file containing venue names and their categories (conf or journal)
venue_category_filename = os.path.join(os.pardir,"app","data","venue_category.csv")

filtered_papers_filename = lambda name,year: os.path.join( "filtered_papers" , 
                                              "{}_{}_filtered_papers.json".format(name,year) )

scores_filename = lambda name,year: os.path.join( "scores" , 
                                              "{}_{}_affiliations.json".format(name,year) )

yearranges = range(2007,2020)

In [24]:
def score_papers(venue,year):
    
    with open(filtered_papers_filename(venue, year), "r") as fh:
        papers = json.load(fh)
        
    
    scores = dict()
    
    citations_counts = dict()
    
    for paper in papers:
        if len(paper["MAG papers"]) == 0:
            continue
            
        mag_paper = max(paper["MAG papers"], key=lambda x: x["CitationCount"])
        
        affiliations = dict()
        
        n_affiliations = len(mag_paper["Affiliations"])
        
        for affiliation in mag_paper["Affiliations"]:
            if affiliation == None:
                continue
            if affiliation not in affiliations:
                affiliations[affiliation] = 1/n_affiliations
            else:
                affiliations[affiliation] += 1/n_affiliations
                
        for aff, weight in affiliations.items():
            if aff not in scores:
                scores[aff] = {"Citation Count": 0, "Publication Count": 0}
                
            scores[aff]["Citation Count"] += weight * mag_paper["CitationCount"]
            scores[aff]["Publication Count"] += weight
        
    with open(scores_filename(venue,year), "w") as fh:
        json.dump(scores, fh)

In [26]:
with open(venue_category_filename, "r") as fh:
    
    reader = csv.reader(fh, delimiter=",")
    
    # skip header row
    next(reader)
    
    for row in reader:
        
        venue_type = row[4]   
        venue = row[0]
        
        try:
            for year in yearranges:
                score_papers(venue,year)
        except:
            print(venue)

3dim
toplas
